# Project outline

Objective: predict % of silica in the iron ore concentrate.

Attention points:
* Data is not sampled in uniform frequency
* Columns 2 and 3 are before feeding into the process (consider retention time)
* Columns 4:8 are relevant quality measurements in the end of the process (probably doesnt need retention time)
* Columns 9:22 are process data
* Last column is % of silica in the iron ore concentrate and is a lab measurement
